In [11]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
918,It may have not been up for academy awards and...,positive
319,"I found this movie really hard to sit through,...",negative
659,"While thinking of ""The Great Escape"" I allowed...",positive
560,I usually like hongkong - martial arts - fanta...,negative
884,Although many audio recordings of great musici...,positive


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [13]:
import nltk
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wadod\AppData\Roaming\nltk_data...


,review,sentiment
918,may academy award admittedly pretty cheesy muc...,positive
319,found movie really hard sit attention kept wan...,negative
659,thinking great escape allowed mind wander back...,positive
560,usually like hongkong martial art fantasy movi...,negative
884,although many audio recording great musician l...,positive


In [14]:
df['sentiment'].value_counts()

sentiment
negative    265
positive    235
Name: count, dtype: int64

In [15]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [16]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
918,may academy award admittedly pretty cheesy muc...,1
319,found movie really hard sit attention kept wan...,0
659,thinking great escape allowed mind wander back...,1
560,usually like hongkong martial art fantasy movi...,0
884,although many audio recording great musician l...,1


In [17]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [18]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/wadoodabdulwadood122010/Text-classification-MLOPS-project.mlflow')
dagshub.init(repo_owner='wadoodabdulwadood122010', repo_name='Text-classification-MLOPS-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\wadod\AppData\Local\Programs\Python\Python38\lib\site-packages\rich\live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1d195144-b37e-4b56-95df-d8033539a1fb&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=0400795e37d8211fe4285ce268250aa62aefe76c34f974360346221b79b96c41




Accessing as wadoodabdulwadood122010

Initialized MLflow to track repo "wadoodabdulwadood122010/Text-classification-MLOPS-project"

Repository wadoodabdulwadood122010/Text-classification-MLOPS-project initialized!

2026/01/14 11:36:19 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/821c8d869dd448bc9510165b0db0ba19', creation_time=1768372579189, experiment_id='0', last_update_time=1768372579189, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-14 11:53:44,057 - INFO - Starting MLflow run...
2026-01-14 11:53:45,188 - INFO - Logging preprocessing parameters...
2026-01-14 11:53:46,181 - INFO - Initializing Logistic Regression model...
2026-01-14 11:53:46,182 - INFO - Fitting the model...
2026-01-14 11:53:46,222 - INFO - Model training complete.
2026-01-14 11:53:46,222 - INFO - Logging model parameters...
2026-01-14 11:53:46,592 - INFO - Making predictions...
2026-01-14 11:53:46,596 - INFO - Calculating evaluation metrics...
2026-01-14 11:53:46,618 - INFO - Logging evaluation metrics...
2026-01-14 11:53:48,357 - INFO - Saving and logging the model...
2026-01-14 11:53:59,027 - INFO - Model training and logging completed in 13.84 seconds.
2026-01-14 11:53:59,027 - INFO - Accuracy: 0.616
2026-01-14 11:53:59,027 - INFO - Precision: 0.6190476190476191
2026-01-14 11:53:59,027 - INFO - Recall: 0.6190476190476191
2026-01-14 11:53:59,027 - INFO - F1 Score: 0.6190476190476191
